<a href="https://colab.research.google.com/github/shake/colab-Llama-2-ipynb/blob/main/huggingface/hg_04_understanding_model_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 小白入门HuggingFace

## 04 看懂模型文件

当使用 `AutoClass` 或者 `pipeline()` 进行模型加载时，我们会看到一组不同的文件被下载。

大家是否好奇过这些都是啥文件，有啥用？今天我们就以模型 [meta-llama/Llama-2-7b-chat-hf](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) 为例，介绍模型的文件组成。

### 一个典型模型加载过程

In [1]:
# make a soft link for  /root/.cache
!mkdir -p /root/.cache/huggingface
!ln -s /root/.cache/huggingface /content

In [ ]:
!pip install git+https://github.com/huggingface/transformers torch sentencepiece --quiet

In [3]:
from transformers import AutoModel

model = AutoModel.from_pretrained("meta-llama/Llama-2-7b-chat-hf", trust_remote_code=True, token="hf_ziljuWSLzXXrl")

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoModel

model = AutoModel.from_pretrained("meta-llama/Llama-2-7b-chat-hf", trust_remote_code=True, use_safetensors=False, token="hf_ziljuWSLzXXrlDvr")

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


#### 模型文件详解

模型包含了如下文件：

| 文件名 | 介绍 |
| --- | --- |
| config.json | 模型架构的主要配置,如 Bert模型设置,预测头部设置,训练参数等。 |
| generation_config.json | 文本生成相关的模型配置。 |
| model-00001-of-00002.safesensors | safesensors文件格式的模型权重参数分块1（见后续介绍） |
| model-00002-of-00002.safesensors | safesensors文件格式的模型权重参数分块2 |
| model.safetensors.index.json | safesensors模型参数文件索引和描述模型切片的 JSON 文件。 |
| pytorch_model-00001-of-00002.bin | pickle序列化的pytorch模型权重参数分块1 |
| pytorch_model-00002-of-00002.bin | pickle序列化的pytorch模型权重参数分块2 |    
| pytorch_model.bin.index.json | pickle序列化的pytorch索引和描述模型切片的 JSON 文件 |
| special_tokens_map.json | tokenizer中特殊标记符(special tokens)到其对应的数字id的映射。|
| tokenizer.json | tokenizer的配置信息,如字典大小,tokenize的策略等。 |
| tokenizer.model | tokenizer的具体模型参数,这是经过训练得到的二进制文件,不可读。 |
| tokenizer_config.json | 使用该tokenizer时的一些配置,如最大序列长度等 |  

一个完整的大型模型来说，通常会被切分成多个碎片(shards)并以 model-00001-of-00002.safetensors 这种命名方式保存。`pytorch_model.bin.index.json` 文件包含所有的模型切片信息,主要包括:
- 模型切片的总数
- 每个切片的元数据,如名称、偏移地址、文件大小等
- 切片如何组合起来重新组成完整模型的说明
- 一些额外的模型信息,如模型名称、框架版本等元数据

`special_tokens_map.json` 包含 `Tokenizer` 特殊标记符（Special Tokens）到其对应的数字ID的映射。

一些常见的特殊标记符定义包括:
1. unk_token - 未登录词(out-of-vocabulary words)的标记id
2. sep_token - 句子分隔的标记id
3. pad_token - 填充序列到相等长度时使用的填充标记id
4. cls_token - 分类任务中使用的分类标记id
5. mask_token - 掩码语言模型任务中使用的掩码标记id


#### safetensors文件

safetensors 是一种安全快速存储和加载tensors的文件格式。通常，`PyTorch` 模型权重会使用Python的 `pickle` 工具将数据序列化到一个 `.bin` 文件中。但是 `pickle` 不安全，`pickle` 的文件可能包含可以执行的恶意代码。`safetensors` 是 `pickle` 的一个安全替代方案，非常适合共享模型权重。

详情请参考 [Using Safetensors](https://huggingface.co/docs/diffusers/main/en/using-diffusers/using_safetensors)